# Dandy Land - OOP Gameplay Simulation
Recently, my son and I dusted off one of our favorite board games to play.  We always play that the youngest player goes first, and it got me thinking: *what kind of advantage, if any, does going first give Player 1?*

In this game, chance rules completely.  Players draw cards and their subsequent move is completely determined, a function only of the card drawn and the player's position on the board (and thus the board itself).  There is only one possible move, in other words, for each player in each turn.  Versions of the gameplay can elicit different outcomes (if, say, you say no two players can occupy the same space on the board at the same time), but with a "fixed" board at all times and the only "variable" being the state of the shuffled deck (which is shuffled once at the start of the game), the result is determined before the first card is drawn.

To quantify the advantage a player has based on their play order, I thought it would be fun to build the game and run a number of simulations, keeping tracking of the winner.  This would give me a chance to take a deeper dive on Python's power as an object-oriented programming langauge, aside from doing very simple statistical analysis to answer my question.  What follows is first my setup of the game and simulations run for games in which 2, 3, and 4 players are active.

#### Imports
I've imported a few packages to help create random shuffling of the deck for each game and to help me test how long simulations will take.  I'll use some of statsmodels functions as well for statistical analysis.

In [4]:
import random
import time
from IPython.display import clear_output
import statsmodels.stats.proportion as sm

#### Building the Board
Building the board starts with creating the structure for the spaces themselves.  
- Space have a `color`, which is what determines if a player lands on them when a card is drawn.\*
- They can also send a player to a new space, so they'll have a `moveto` attribute.
- Spaces can also cause a player to skip a turn, so they'll hold a `skip` attribute as well.

\*In the case of spaces that are indicated by a character, which can be jumped to if a card with that character's symbol is drawn, the color will be the indicator of that particular space. 

In [6]:
class Space(object):
    '''
    A Space intance represents a single space on the Board
    '''
    def __init__(self,color,moveto,skip):
        self.color = color
        self.moveto = moveto
        self.skip = skip

The board itself will be an ordered collection of spaces.  The Space objects could also hold an attribute for their space "number", but this unecessary if the board "builds" the order itself. I chose to use a dictionary to house the spaces, keyed to its space number.  The first space is the 0 space and is where everyone starts.  The final space in the game can be accessed by drawing any color, so it also has a unique identifier of "all" for its color.

In [8]:
class Board(object):
    '''
    A Board instance containts Space instances
    For our version of the game, the Board (and thus the order of the Space instnaces and
    their attributes does not change.
    '''
    def __init__(self):
        self.spaces = {0:Space(None,None,False),
                       1:Space('red',None,False), # The first space (i.e., space number 1) on the board is red
                       2:Space('purple',None,False),
                       3:Space('yellow',None,False),
                       4:Space('blue',36,False),
                       5:Space('orange',None,False),
                       6:Space('green',None,False),
                       7:Space('red',None,False),
                       8:Space('purple',None,False),
                       9:Space('yellow',None,False),
                       10:Space('blue',None,False),
                       11:Space('orange',None,False),
                       12:Space('green',None,False),
                       13:Space('red',None,False),
                       14:Space('purple',None,False),
                       15:Space('yellow',None,False),
                       16:Space('blue',None,False),
                       17:Space('orange',None,False),
                       18:Space('green',25,False),
                       19:Space('red',None,False),
                       20:Space('purple',None,False),
                       21:Space('A',None,False), # This is a special space, accessed only if its card is drawn
                       22:Space('yellow',None,False),
                       23:Space('blue',None,False),
                       24:Space('orange',None,False),
                       25:Space('green',None,False),
                       26:Space('red',None,False),
                       27:Space('purple',None,True),
                       28:Space('yellow',None,False),
                       29:Space('blue',None,False),
                       30:Space('orange',None,False),
                       31:Space('green',None,False),
                       32:Space('red',None,False),
                       33:Space('B',None,False),
                       34:Space('purple',None,False),
                       35:Space('yellow',None,False),
                       36:Space('blue',None,False),
                       37:Space('orange',None,False),
                       38:Space('green',None,False),
                       39:Space('red',None,False),
                       40:Space('purple',None,False),
                       41:Space('yellow',None,False),
                       42:Space('blue',None,False),
                       43:Space('orange',None,False),
                       44:Space('C',None,False),
                       45:Space('red',None,False),
                       46:Space('purple',None,False),
                       47:Space('yellow',None,False),
                       48:Space('blue',None,False),
                       49:Space('orange',None,False),
                       50:Space('green',None,False),
                       51:Space('red',None,False),
                       52:Space('D',None,False),
                       53:Space('purple',None,False),
                       54:Space('yellow',None,True),
                       55:Space('blue',None,False),
                       56:Space('orange',None,False),
                       57:Space('green',None,False),
                       58:Space('red',None,False),
                       59:Space('purple',None,False),
                       60:Space('yellow',None,False),
                       61:Space('blue',None,False),
                       62:Space('orange',None,False),
                       63:Space('green',None,False),
                       64:Space('red',None,False),
                       65:Space('purple',None,False),
                       66:Space('yellow',None,False),
                       67:Space('E',None,False),
                       68:Space('blue',None,False),
                       69:Space('orange',None,False),
                       70:Space('green',None,False),
                       71:Space('red',None,False),
                       72:Space('purple',None,False),
                       73:Space('yellow',None,False),
                       74:Space('blue',None,False),
                       75:Space('orange',None,False),
                       76:Space('green',None,False),
                       77:Space('red',None,False),
                       78:Space('purple',None,False),
                       79:Space('yellow',None,False),
                       80:Space('blue',None,False),
                       81:Space('orange',None,False),
                       82:Space('green',None,False),
                       83:Space('all',None,False), # The final space on the board, accessed by any color card
                      }

    def get_colorlist(self):
        '''
        This function will return a list of the colors on the board.
        It will be helpful in finding the next space on the board open 
        to a player sitting on a particular colored space.
        param: Board object
        returns: list of colors from the Board
        '''
        spaces = self.spaces.values()
        spaces_colors = [space.color for space in spaces]
        return spaces_colors

#### Building the Deck
Building the deck starts with creating the structure for the cards in the deck.  
- Cards have a `color`, which can also be a character symbol.
- The Cards can also indicate how many colors to traverse, either one or two, so there will be a `moves` attribute.
- Cards can also send a player forward in the case of colors that are drawn or in any direction if a character symbol is drawn, so there is an attribute related to the diretion, `direct`, that a Card can send a player.

In [10]:
class Card(object):
    '''
    A Card intance represents a single card within the Deck
    '''
    def __init__(self,color,moves,direct):
        self.color = color
        self.moves = moves
        self.direct = direct

The Deck is comprised of 44 cards.  Each of the colors has at least three cards for single moves and three for doulbe moves.  Yellow, green, and orange have an extra color card for a single move.  There are also 5 character cards.

In [12]:
class Deck(object):
    '''
    A Deck instance contains Card instances (in a list).
    For our version of the game, the Deck is fixed in terms of its contents, 
    but it can be re-ordered (shuffled)
    '''
    def __init__(self):
        cards = []
        colors = ['red','orange','yellow','green','blue','purple']
        move_opts = [1,1,1,2,2,2]
        # Cycling through the colors and moves in two loops creates most of the cards
        for color in colors:
            for move in move_opts:
                cards.append(Card(color,move,'forward'))
        # We can add the three extra color cards from the game
        cards.append(Card('orange',1,'forward'))
        cards.append(Card('yellow',1,'forward'))
        cards.append(Card('green',1,'forward'))
        # And also add the five character cards
        jumps = [Card('A',1,'any'),Card('B',1,'any'),
                 Card('C',1,'any'),Card('D',1,'any'),
                 Card('E',1,'any')]
        cards.extend(jumps)
        self.cards = cards

    def shuffle(self):
        '''
        This function will update a Deck in a a re-ordered (shuffled) state.
        It will be helpful in creating games with "random" initial conditions.
        param: Deck object
        returns: None
        '''
        self.cards = sorted(self.cards, key=lambda x:random.random())
    
    def draw(self):
        '''
        This function will return from a deck the identity of the card "on top", 
        and it will update the deck itself to keep the same order save for moving the drawn card
        to the "bottom" of the deck (and all other cards shift "up" one).
        '''
        drawn_card = self.cards[0]
        remaining_deck = self.cards[1:]
        self.cards = remaining_deck+[drawn_card]
        return drawn_card

    def draw_check(self):
        '''
        This is a test funciton that simply returns (reveals) the first card "on top of" the Deck.
        '''
        return self.cards[0]

#### Building the Game
Building the game begins with create players.  Players will have a number of attributes:
- Players have a `number`, which inidicates if they are Player **1**, Player **2**, etc.
- The Players will occupy a `space` on the board.
- Players may have landed on a card that causes them to skip their next turn, and we can indicate this with a boolean indicator for `skip`.  
- We can also track how many cards they have `drawn` in a given game too.

In [14]:
class Player(object):
    '''
    A Player intance represents a player of the Game.
    '''
    def __init__(self,number):
        self.number = number
        self.space = 0
        self.skip = False
        self.drawn = 0

The game itself is fairly straightforward in terms of rules.  We can initialize the game to contain the `Board`, a `Deck` (shuffled), and `Players` so it'll have attributes related to each of these objects (with an added attribute that simply notes the number of players \[`no_players`\].  We can also keep track of the number of cards `drawn` in total.

Moves are taken in order, and during each move, a player draws a card and takes the appropriate action.  The only wrinkle that the game rules indicate are that if a player is set to land on a colored space that is occupied, they move to the next space of that color.  A player will be skipped and not draw a card if they had landed on a 'skip' space in their last turn.  The game is played, cycling through each player in order, until a player lands on the final space.

In [16]:
class Game(object):
    '''
    A Game instance contains a Board, a Deck (which we immediately shuffle), and a dictonary of Players,
    keyed to their player number.
    '''
    def __init__(self,no_players):
        self.board = Board()
        self.deck = Deck()
        self.deck.shuffle()
        self.no_players = no_players
        players = dict()
        for i in range(no_players):
            players[i+1] = Player(i+1)
        self.players = players
        self.drawn = 0
    
    def move(self,player):
        '''
        This function completes a 'move' for the player that is passed in.  It draws a card and determines
        what to do; it updates the attributes for skip and space for the player, return a boolean as to whether
        or not the player finished the game in their turn.  This method also updates the Game's Deck as cards
        are drawn.
        param: Player object
        returns: boolean as to whether or not the player moved won the game
        '''
        card = self.deck.draw() # Gets a card from the deck
        player.drawn += 1 # update the number of cards drawn for the player
        self.drawn += 1 # update the number of cards drawn for the game
        for i in range(card.moves): # Move 1 or 2 spaces, depending on type of card
            space_opts = self.get_openSpaces() # Finds out the status of the spaces on the board
            if card.color in ['red','orange','yellow','green','blue','purple']:
                space_opts = space_opts[player.space+1:] # For color cards drawn, player is limited to moving forawrd
                try:
                    i = space_opts.index(card.color) # moves the player to the color space it can
                    player.space += i+1
                except:
                    return True # if the player can only move to the final space, they win!
            else:
                player.space = space_opts.index(card.color) # For character cards, move the player to that space
        player.skip = self.board.spaces[player.space].skip # Assign player a 'True' skip attribute if they landed on a skip
        moveto_space = self.board.spaces[player.space].moveto # Find out if they landed on a space that advances them further
        if moveto_space:
            player.space = moveto_space # If the spot they landed on advances them further, execute that
        return False # Indicates the player did not win and the game continues
            
    def get_nextPlayer(self,player):
        '''
        This function determines who the next player is based on a passed in player object
        param: Player object whose turn it is
        returns: The next Player (object) whose turn it is
        '''
        i = player.number
        if i == self.no_players:
            return self.players[1]
        else:
            return self.players[i+1]
    
    def get_openSpaces(self):
        '''
        This method gets the state of the board and updates it to determine which colors are open
        param: None
        returns: A list of the colors on the board, accounting for which spaces are occupied and thus not open.
        '''
        all_spaces = self.board.get_colorlist()
        for player in self.players.values():
            all_spaces[player.space] = "occupied"
        return all_spaces
            
    def play(self):
        '''
        This method moves through each player by completing moves for each of them, ending only when 
        one player wins on their turn.  It then returns the number for the player.
        param: None
        returns: The winning player's number.
        '''
        winner = False
        player = self.players[game.no_players]
        while not winner:
            player = game.get_nextPlayer(player)
            if player.skip:
                player.skip = False
                player = game.get_nextPlayer(player)
                continue
            winner = self.move(player)
        return player.number

#### Simulations
Now that we have the game set up, we can start to run simulations.  I'll start with two-player games, and I'll run through 10,000,000 games, keeping track of the number of time each player (Player 1 or Player 2) wins.

In [18]:
no_players = 2
vals = [0]*no_players
results_2 = dict(zip(range(1,no_players+1),vals))
total_drawn = []
winner_drawn = []
no_games_2 = 1_000_000
start_time = time.time()
for i in range(no_games_2):
    # if i % (no_games_2//min(no_games_2,100)) == 0:
    #     clear_output()
    #     print(f'Game {i} of {no_games_2}')
    game = Game(no_players)
    winner = game.play()
    results_2[winner] = results_2[winner] + 1
    total_drawn.append(game.drawn)
    winner_drawn.append(game.players[winner].drawn)
ave_game_drawn = sum(total_drawn)/len(total_drawn)
ave_winner_drawn = sum(winner_drawn)/len(winner_drawn)
end_time = time.time()
total_time_2 = round((end_time-start_time)/60,1)
clear_output()
print(f'Time to run {no_games_2} game simulations with {no_players} players: {total_time_2} minutes')
print('Player Number wins:',results_2)
print('Average cards drawn in game:',ave_game_drawn)
print('Average cards drawn by winner:',ave_winner_drawn)

Time to run 1000000 game simulations with 2 players: 2.5 minutes
Player Number wins: {1: 523508, 2: 476492}
Average cards drawn in game: 19.897468
Average cards drawn by winner: 10.210488


Player 1 wins about 52.3% of the time in a two-player game.  Since we ran a massive number of simulations, this is going to be very close to the expected winning percentage for Player 1 for this particular game.  But, for the sake of exercise, we can run this through it's paces and set up a true test of a null hypothesis that there is no advantage to being Player 1.  To whit:

- The null hypothesis is that the expected win percentage for Player 1 is less than or equal to 50% (0.5).  
- The alternative hypothesis is that that is some advantage, so that Player 1 would be expected to win more than half the time.  
- We are not considering if there is a disadvantage to going first, so this will only look at half of the distribution and thus we'll consider a one-sided p-value.

In [20]:
x_2 = results_2[1]
p_null_2 = 0.5
z_stat_2, p_value_2 = sm.proportions_ztest(x_2, no_games_2, p_null_2, alternative='larger')
print("Z-statistic:", z_stat_2)
print("P-value:", p_value_2)

Z-statistic: 47.068050843922705
P-value: 0.0


We can see that the proportion value from our simulation (52.3%) lies nearly 50 standard deviations away from the null value, and thus the p-value (one-sided) is effectively zero.  In other words, there is strong evidence to reject the null hypothesis that there is no advantage to going first.  The data in fact suggests that the player that goes first in a two-player game is likely to win 52.3% of the time.